# Monitoring
## Watchers
The Monitor class allows you to track defined parameters, visualize their time series, and implement custom reactions if they go outside of defined thresholds. Let's say we have some function which measures and returns a voltage, which for the purposes of this tutorial we will generate from a normal distribution:

In [ ]:
import numpy as np
def read_voltage():
    return np.random.normal(1, 0.05)

It's very simple to define a Monitor to watch this voltage:

In [ ]:
from vigilant import Monitor
m = Monitor()
m.watch(read_voltage)
m.start(period=1)

The periodic acquisiton uses Python's sched module for event scheduling to avoid accumulated drifts. For example, if your check() method takes 50 ms to return and you call it in a loop with a 1 s delay, the true time between checks will actually be 1.05 s. Using the scheduler allows drift-free checking; however, you should make sure that the check() call always returns in a time shorter than your chosen period!

### Plotting
Interactive realtime plotting in Jupyter notebooks is enabled by attaching a Visualizer to the Monitor:

In [ ]:
from vigilant.extensions import Visualizer
m.add_extension(Visualizer())

### Watching new variables
New variables can be added dynamically while the monitor is running:

In [ ]:
def read_current():
    return np.random.normal(0.1, 0.01)
m.watch(read_current, name='current')

## Listeners
You can also define passively-monitored objects - instead of being queried by the Monitor for their state, the Monitor can subscribe to a remote data source to be updated asynchronously with the monitoring cycle: 

In [ ]:
m.listen('data feed', address='127.0.0.1:9000')

Let's send random data into this feed every 250 ms:

In [ ]:
from vigilant.extensions import Publisher
from threading import Thread 
import time 

feed = Publisher('127.0.0.1:9000')

def generate_data(feed):
    while True:
        feed.update(np.random.normal(0.3, 0.05))
        time.sleep(0.25)
        
Thread(target=generate_data, args=(feed,)).start()

Notice that the plot is only updated once per second - this is because the feed is checked every 250 ms in a separate process and the result is put into a queue, which is read during the normal monitoring cycle.

## Reactions

When you specify a variable to watch, you can pass a threshold tuple defining the "good" range for the variable. You can also pass a function to be called when the variable goes outside of the threshold:

In [ ]:
def read_noisy_voltage():
    return np.random.normal(0.5, 0.1)

def alert():
    print('Noisy voltage below threshold!')
    
m.watch(read_noisy_voltage, threshold=(0.4, None), reaction = alert)

Reactions for Listeners can be set as well; the logical comparison will be made during the monitoring cycle, not in the asychronous listening process.

To stop the monitoring, call the Monitor.stop() method:

In [ ]:
m.stop()

## Triggering
Monitoring can be synced to some other process through software triggers, e.g. TTL pulses acquired with a DAQ board. Just define a method which returns as soon as a trigger is received and pass the trigger into the Monitor.start() method. Let's simulate a trigger arriving once per second:

In [ ]:
import time
def trigger():
    time.sleep(1)
    return

m.start(trigger=trigger)

In [ ]:
m.stop()

# Extensions
By default, Vigilant runs in a very lightweight mode which only acquires data and stores it internally. More sophisticated behavior can be added by registering an extension class using the ``Monitor.add_extension()`` method. As well as the supported classes, you can create custom extensions - the only requirement is that they have an ``update(data)`` method which takes a pandas DataFrame and processes it however you'd like.

## ZMQ feed
A ZeroMQ-based pub-sub protocol can be implemented by attaching a Publisher object to the Monitor:

In [ ]:
from vigilant.extensions import Publisher
m.add_extension(Publisher('127.0.0.1:1107'))

The Publisher will broadcast each new measurement on the specified address and port. To receive these messages, you can create a Subscriber:

In [ ]:
from vigilant.extensions import Subscriber
sub = Subscriber('127.0.0.1:1107')
sub.receive()

This can be used to broadcast data to other processes for analysis.

## TICK database
For long-term monitoring with large datasets, the pandas + ZeroMQ stack can be replaced with an InfluxDB database. The InfluxClient class can be used for this purpose:

In [ ]:
from vigilant.extensions import InfluxClient
m.add_extension(InfluxClient(address='127.0.0.1:8086', database='demo2'))

The same client class can be used to retrieve data from other scripts on any networked PC:

In [ ]:
other_client = InfluxClient(address='127.0.0.1:8086', database='demo2')
other_client.read()